# Data collection - scrap work
This also shows you how to get an access key.

In [ ]:
import requests
import json
from config import *
from datetime import datetime
from dateutil.relativedelta import relativedelta
from time import mktime
import time

# Get recent tracks from Last FM

In [ ]:
data = requests.get("http://ws.audioscrobbler.com/2.0", 
                    {"method": "user.getrecenttracks", 
                     "user": "tysonpo", 
                     "api_key": lastfm_client_id, 
                     "format": "json"}).json()
last_track = data["recenttracks"]["track"][0] # most recently played track
last_track

In [ ]:
track, artist, album, date = last_track["name"], last_track["artist"]["#text"], last_track["album"]["#text"], last_track["date"]["uts"]
track, artist, album, date

# Get each track's information from Spotify

## Authorization

In [ ]:
url = "https://accounts.spotify.com/authorize?client_id=%s&response_type=code&redirect_uri=%s" % (spotify_client_id, spotify_redirect_uri)
print(url)

# click on url. The string following "code=" is your authorization code -- copy and paste it into the cell below

In [ ]:
auth_code = "..."

In [ ]:
# get access token
data = requests.post('https://accounts.spotify.com/api/token', 
                  {'code': auth_code, 'redirect_uri': spotify_redirect_uri, 'grant_type': "authorization_code", 
                   "client_id": spotify_client_id, "client_secret": spotify_client_secret}).json()
spotify_access_token = data["access_token"]

In [ ]:
spotify_access_token

In [ ]:
spotify_headers = {"client-id":spotify_client_id, "Authorization":"Bearer %s" % spotify_access_token}

## Get its ID

In [ ]:
data = requests.get("https://api.spotify.com/v1/search", 
                    {"q": track + " " + artist, "type": "track", "limit":50}, 
                    headers=spotify_headers).json()
data

In [ ]:
matches = [x for x in data["tracks"]["items"] if x["album"] and x["album"]["name"] == album] # get all tracks with correct album name
ID = max(matches, key = lambda x: x["popularity"])["id"] # if multiple, get one with with highest popularity. Get the tracks id
ID

## Get audio features 
We can get audio features for multiple tracks by concatenating multiple track IDs in a comma-separated list

In [ ]:
tracks = ['1y4Kln6VEjQMpmHW7j9GeY', '7Lf7oSEVdzZqTA0kEDSlS5', '1kdiiFGX1Htx0aVZYaDwEJ', '7xbWAw3LMgRMn4omR5yVn3', '6RUKPb4LETWmmr3iAEQktW']
data = requests.get("https://api.spotify.com/v1/audio-features",
                    {"ids": ",".join(tracks)},
                    headers=spotify_headers).json()["audio_features"]
data = [x for x in data if x] # remove null results
drop_keys = ["type", "uri", "track_href", "analysis_url"]
data = [{key:dct[key] for key in dct if key not in drop_keys} for dct in data]
data